In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
data_dir = '/kaggle/input/poultry/dataset2/poultry_diseases'
dataset = datasets.ImageFolder(data_dir, transform=transform)
print(f"Classes: {dataset.classes}")


In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)  # smaller batch for pi
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [ ]:
print(f"Train: {len(train_data)}, Test: {len(test_data)}")

In [ ]:
model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(dataset.classes))
model = model.to(device)

In [ ]:
# training stuff
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# train function
def train():
    model.train()
    for epoch in range(5):  # fewer epochs
        total_loss = 0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            if i % 10 == 0:  # print less often
                print(f'Epoch {epoch+1}, Batch {i}, Loss: {loss.item():.3f}')
        
        acc = 100 * correct / total
        print(f'Epoch {epoch+1}: Accuracy = {acc:.2f}%, Loss = {total_loss/len(train_loader):.3f}')


In [ ]:
# test function
def test():
    model.eval()
    correct = 0
    total = 0
    predictions = []
    actual = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            predictions.extend(predicted.cpu().numpy())
            actual.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy, actual, predictions


In [ ]:

# simple evaluation - just show some results
def evaluate():
    model.eval()
    dataiter = iter(test_loader)
    images, labels = next(dataiter)
    
    with torch.no_grad():
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
    
    # show first 4 predictions
    for i in range(4):
        actual_class = dataset.classes[labels[i]]
        pred_class = dataset.classes[predicted[i]]
        status = "✓" if actual_class == pred_class else "✗"
        print(f"{status} Actual: {actual_class}, Predicted: {pred_class}")


In [ ]:
print("Training started...")
train()

In [ ]:
print("\nTesting...")
test_acc, y_true, y_pred = test()

In [ ]:
print("\nSample predictions:")
evaluate()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

In [ ]:
class_names = ['healthy', 'unhealthy']  

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
print("\n" + "="*50)
print("ADDITIONAL EVALUATIONS")
print("="*50)

from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

# Get class names from train_loader
class_names = train_loader.dataset.dataset.classes

print("\nPer-class results:")
for i, class_name in enumerate(class_names):
    print(f"{class_name}: Precision={precision[i]:.2f}, Recall={recall[i]:.2f}, F1={f1[i]:.2f}")

In [ ]:
#save model as .pt file for raspberry pi
torch.save(model, 'poultry_model.pt')  # saves entire model
print(f"\nModel saved as poultry_model.pt")

In [ ]:
# also save just weights if needed
torch.save(model.state_dict(), 'poultry_weights.pt')
print("Weights saved as poultry_weights.pt")